In [35]:
import sys
sys.path.append("../")
import API_keys

In [ ]:
import sounddevice as sd
import soundfile as sf
from openai import OpenAI

client = OpenAI()

response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input="tôi có thể giúp gì cho bạn.",
)
response.write_to_file("speak.mp3")
# response.with_streaming_response.method("speak.mp3")

file_path = "speak.mp3"
def play_wav(file_path):
    data, fs = sf.read(file_path, dtype='float32')
    sd.play(data, fs)
    sd.wait()
play_wav(file_path)

In [33]:
import wave
def reset_audio(file):
    # Open the input WAV file for reading
    with wave.open(file, 'rb') as input_wav:
        # Get the parameters of the input WAV file
        params = input_wav.getparams()
        
        # Create an empty audio data string
        empty_audio_data = b''
        
        # Open the output WAV file for writing
        with wave.open(file, 'wb') as output_wav:
            # Set the parameters for the output WAV file
            output_wav.setparams(params)
            
            # Write the empty audio data to the output file
            output_wav.writeframes(empty_audio_data)
reset_audio('prompt.wav')

In [18]:
import pyttsx3
engine = pyttsx3.init()
engine.setProperty('voice', 'vietnam')
engine.say('xin chào mọi người')
engine.runAndWait()

In [ ]:
for voice in engine.getProperty('voices'):
    print(voice)

In [13]:
import webrtcvad
import pyaudio
import wave

def record_audio():
    # Initialize PyAudio and VAD
    pa = pyaudio.PyAudio()
    vad = webrtcvad.Vad(3)  # Level 3 is more aggressive, detecting shorter segments of silence

    # Parameters for the audio stream
    format = pyaudio.paInt16
    channels = 1
    rate = 16000
    chunk_duration_ms = 30  # 30ms
    chunk_size = int(rate * chunk_duration_ms / 1000)

    # Open audio stream
    stream = pa.open(format=format, channels=channels, rate=rate,
                     input=True, frames_per_buffer=chunk_size)

    print("Listening...")

    frames = []
    num_silent_chunks = 0
    max_silent_chunks = 30  # Adjust this to change the sensitivity

    while True:
        chunk = stream.read(chunk_size)
        is_speech = vad.is_speech(chunk, rate)

        if is_speech:
            frames.append(chunk)
            num_silent_chunks = 0
        else:
            num_silent_chunks += 1
            print(num_silent_chunks)
            if num_silent_chunks > max_silent_chunks:
                break

    print("Finished recording")

    stream.stop_stream()
    stream.close()
    pa.terminate()

    # Save the recorded audio to a file
    wf = wave.open("audio/wake_detect.wav", 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(pa.get_sample_size(format))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

if __name__ == "__main__":
    record_audio()


Listening...
1
1
2
3
1
2
1
2
3
1
2
3
4
1
2
3
4
5
6
7


KeyboardInterrupt: 

In [34]:
from openai import OpenAI
client = OpenAI()

audio_file= open("prompt.wav", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)
print(transcription.text)

BadRequestError: Error code: 400 - {'error': {'message': 'Audio file is too short. Minimum audio length is 0.1 seconds.', 'type': 'invalid_request_error', 'param': 'file', 'code': 'audio_too_short'}}

In [ ]:
porcupine_demo_mic --access_key rfh6u55ROt55M7g021wT0YnW5bvC1Y9k3iyptQ8R09HxpQPN6wtyIw== --keywords porcupine

In [1]:
import pvporcupine

porcupine = pvporcupine.create(
  access_key='rfh6u55ROt55M7g021wT0YnW5bvC1Y9k3iyptQ8R09HxpQPN6wtyIw==',
  keywords=['picovoice', 'bumblebee']
)

In [7]:
def get_next_audio_frame():
  pass


while True:
    audio_frame = get_next_audio_frame()
    keyword_index = porcupine.process(audio_frame)
    if keyword_index == 0:
       print('porcupine')
    elif keyword_index == 1:
       print('bumblebee')


TypeError: object of type 'NoneType' has no len()

In [18]:
import pyaudio
import webrtcvad
import threading
import time
# from porcupine import Porcupine
import pvporcupine

class VoiceAssistant:
    def __init__(self):
        self.vad = webrtcvad.Vad(3)
        self.pa = pyaudio.PyAudio()
        self.rate = 16000
        self.channels = 1
        self.chunk_duration_ms = 16
        self.chunk_size = int(self.rate * self.chunk_duration_ms / 1000)
        self.max_silent_chunks = 15
        self.sleep_timer = None
        self.in_sleep_mode = True
        self.porcupine = pvporcupine.create(
  access_key=API_keys.porcupine_key,
  keywords=['hey siri', 'bumblebee']
)
        self.stream = self.pa.open(format=pyaudio.paInt16, channels=self.channels, rate=self.rate,
                                   input=True, frames_per_buffer=self.chunk_size)
        
    def listen(self):
        print("Listening...")
        frames = []
        num_silent_chunks = 0

        while True:
            chunk = self.stream.read(self.chunk_size)
            is_speech = self.vad.is_speech(chunk, self.rate)
            
            if is_speech:
                frames.append(chunk)
                num_silent_chunks = 0
                self.reset_sleep_timer()
            else:
                num_silent_chunks += 1
                if num_silent_chunks > self.max_silent_chunks:
                    self.start_sleep_timer()
                    break

        return b''.join(frames)
    
    def start_sleep_timer(self):
        if self.sleep_timer:
            self.sleep_timer.cancel()
        self.sleep_timer = threading.Timer(5, self.enter_sleep_mode)
        self.sleep_timer.start()

    def reset_sleep_timer(self):
        if self.sleep_timer:
            self.sleep_timer.cancel()
            self.sleep_timer = None

    def enter_sleep_mode(self):
        self.in_sleep_mode = True
        print("Entering sleep mode. Say the wake word to activate.")
        self.detect_wake_word()

    def detect_wake_word(self):
        while self.in_sleep_mode:
            pcm = self.stream.read(self.chunk_size)
            print(pcm)
            keyword_index = self.porcupine.process(pcm)
            print(keyword_index)
            if keyword_index >= 0:
                print('porcupine')
                # self.in_sleep_mode = False
                print("Wake word detected. Exiting sleep mode.")
                break

    def run(self):
        self.detect_wake_word()
        while True:
            if not self.in_sleep_mode:
                audio_data = self.listen()
                
                # print(keyword_index)
                # Process the audio data, e.g., transcribe and respond
                print("Processing user input...")
            else:
                time.sleep(0.1)  # Sleep briefly to prevent busy-waiting

    def close(self):
        self.stream.stop_stream()
        self.stream.close()
        self.pa.terminate()
        self.porcupine.delete()

if __name__ == "__main__":
    assistant = VoiceAssistant()
    try:
        assistant.run()
    except KeyboardInterrupt:
        assistant.close()


b"\x00\x00\xff\xff\x03\x00\xf9\xff\n\x00\xfa\xff\xf1\xffK\x001\xfe\xd2\xfc\xe0\xfc\x8f\xfc\r\xfeB\xfe\xec\xfc<\xfd\x01\xfeu\xfe,\xff\xee\xffp\x00\xd5\x00\x0f\x01\x15\x00\x1d\x009\x01\xcb\x01\x94\x02\x0f\x03\xc7\x02\xd1\x01E\x01(\x01=\x01\xe8\x01\x9a\x02\xa6\x02\xb1\x01\x0b\x01\x97\x01G\x02J\x02o\x01'\x01\x99\x01\xc4\x01;\x02<\x02]\x02\xad\x02\x18\x02\xa5\x02\xf4\x039\x04r\x03\xcf\x03\xb8\x03\x85\x01\x95\x01F\x01\xb8\xffJ\xff\xd2\xfe=\xff\x82\x00<\x01h\x02V\x02\xee\x00D\x00\xab\x00\x9f\x01\xe2\x01\xeb\x014\x02\xc3\x01Z\x00l\x00\xc0\x00|\x01e\x01]\x00\xfe\xff\xc3\x00U\x01\x9f\x00<\x01E\x01\xcf\x01\xa0\x02*\x029\x02\x93\x03\x8b\x03\x7f\x02\x16\x03\x0b\x03\xff\x02\x8c\x03\xa0\x04\x01\x04\xaf\x03g\x05\xa4\x05/\x05\x0f\x04\xce\x04\xc3\x04\xd6\x03\x06\x04\xf0\x03\x9a\x04d\x03W\x03\x14\x04)\x03\xae\x025\x02\x1e\x01\xeb\x00\x15\x024\x02\x1a\x02X\x02X\x02~\x02\x02\x03\xe9\x03\x1c\x03F\x02\xcc\x02\xeb\x02\x9b\x02\xf9\x02\xdc\x03\xa5\x03\xe9\x02\x00\x03.\x03\x9e\x02K\x02\xb4\x02>\x03\xa8\x02\xcd\x

In [9]:
pcm = "\x00\x00\xff\xff\x03\x00\xf9\xff\n\x00\xfa\xff\xf1\xffK\x001\xfe\xd2\xfc\xe0\xfc\x8f\xfc\r\xfeB\xfe\xec\xfc<\xfd\x01\xfeu\xfe,\xff\xee\xffp\x00\xd5\x00\x0f\x01\x15\x00\x1d\x009\x01\xcb\x01\x94\x02\x0f\x03\xc7\x02\xd1\x01E\x01(\x01=\x01\xe8\x01\x9a\x02\xa6\x02\xb1\x01\x0b\x01\x97\x01G\x02J\x02o\x01'\x01\x99\x01\xc4\x01;\x02<\x02]\x02\xad\x02\x18\x02\xa5\x02\xf4\x039\x04r\x03\xcf\x03\xb8\x03\x85\x01\x95\x01F\x01\xb8\xffJ\xff\xd2\xfe=\xff\x82\x00<\x01h\x02V\x02\xee\x00D\x00\xab\x00\x9f\x01\xe2\x01\xeb\x014\x02\xc3\x01Z\x00l\x00\xc0\x00|\x01e\x01]\x00\xfe\xff\xc3\x00U\x01\x9f\x00<\x01E\x01\xcf\x01\xa0\x02*\x029\x02\x93\x03\x8b\x03\x7f\x02\x16\x03\x0b\x03\xff\x02\x8c\x03\xa0\x04\x01\x04\xaf\x03g\x05\xa4\x05/\x05\x0f\x04\xce\x04\xc3\x04\xd6\x03\x06\x04\xf0\x03\x9a\x04d\x03W\x03\x14\x04)\x03\xae\x025\x02\x1e\x01\xeb\x00\x15\x024\x02\x1a\x02X\x02X\x02~\x02\x02\x03\xe9\x03\x1c\x03F\x02\xcc\x02\xeb\x02\x9b\x02\xf9\x02\xdc\x03\xa5\x03\xe9\x02\x00\x03.\x03\x9e\x02K\x02\xb4\x02>\x03\xa8\x02\xcd\x01&\x02\xd3\x01\x85\x01\xfe\x01A\x02\x1f\x02\xd4\x01\x1a\x02C\x02\xb8\x01\xee\x00\xc0\x00)\x00\xc3\xff\xb7\xff\xa1\xfeU\xfe\xaf\xfeq\xfeF\xfe\xe1\xfd\x08\xfe\x95\xfe\x9b\xfe\xc7\xfeL\xff\xa1\xff\t\xff\xc3\xfe\x17\xff\xbd\xfe\xeb\xfe\xcd\xfe_\xfe\x00\xfe\x13\xfd\xeb\xfc\x03\xfd\x05\xfd|\xfcm\xfb\x8e\xfa6\xfaB\xfa\x16\xfa\xb8\xfa\xb5\xfb\xd5\xfb\xb1\xfb\x19\xfc7\xfd\x0b\xfd\xc6\xfc\x04\xfdl\xfc\x1a\xfd\x85\xfe\x84\xfe\xfb\xfdo\xfd\x85\xfc\x81\xfb\xec\xfa\x8f\xfbJ\xfbt\xfax\xfb[\xfbh\xfb[\xfc\x82\xfc\x1b\xfc\x1a\xfc\xc7\xfb\x07\xfb\xd6\xfbl\xfb\x85\xfb\xc1\xfb\xe9\xfa\xf4\xfb\xb1\xfc\x81\xfc\xc5\xfc\xda\xfb=\xfbz\xfb&\xfb]\xfb}\xfbY\xfb\xe8\xfbg\xfc\x8a\xfc\xc8\xfb\xe6\xfb\x90\xfc\xd6\xfb\x91\xfb\xb9\xfc\xf5\xfc\xd0\xfc%\xfd.\xfdy\xfd\xb9\xfc\xc6\xfb\xa2\xfb\x93\xfb$\xfc\xc3\xfc"
# porcupine.process(pcm)
"d2\xfb2\xfb\x03\xfb\xf6\xfbe\xfdo\xfe" in pcm

False

#PocketSphinx

In [24]:
import os
from pocketsphinx import LiveSpeech
import pyaudio
import wave
def record_audio():
    """Record audio for a fixed duration or until silence is detected."""
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    CHUNK = 1024
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "output.wav"

    audio = pyaudio.PyAudio()

    # start Recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
    print("Recording...")
    frames = []

    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("Finished recording.")

    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

model_path = "/home/xuanai/LLM/Bot/retrieval-text/sphinx_models"

speech = LiveSpeech(
    lm=False,
    keyphrase='hey assistant',  # Replace with your wake word
    kws_threshold=1e-20,
    hmm=os.path.join(model_path, 'cmusphinx-en-us-5.2'),
    dict=os.path.join(model_path, 'cmudict-en-us.dict')
)

for phrase in speech:
    print("Wake word detected!")
    record_audio()
    # Your logic to handle wake word detection


In [7]:
import os
from pocketsphinx import Decoder, Config
import pvporcupine
import sys
sys.path.append("../")
from API_keys import *
wake_words = ['porcupine', 'bumblebee', 'jarvis', 'alexa']
porcupine = pvporcupine.create(
  access_key=porcupine_key,
  keywords=wake_words
)

# Path to the sphinx models directory
# model_path = 'sphinx_models'  # Update with the correct path to your models

# # Configuration for the decoder
# config = Config(
#     hmm=os.path.join(model_path, 'cmusphinx-en-us-5.2'),  # Path to the acoustic model
#     dict=os.path.join(model_path, 'cmudict-en-us.dict'),  # Path to the dictionary file
#     keyphrase='hey assistant',  # Replace with your wake word
#     kws_threshold=1e-20  # Set the threshold for keyword spotting
# )

# # Create a decoder with the specified configuration
# decoder = Decoder(config)

# Path to your recorded audio file
audio_file_path = 'audio/wake_detect.wav'  # Replace with the path to your recorded audio file

# Open the audio file and decode it
with open(audio_file_path, 'rb') as audio_file:
    # decoder.start_utt()
    while True:
        buf = audio_file.read(512)
        if len(buf)==512:
            # decoder.process_raw(buf, False, False)
            key = porcupine.process(buf)
            if key>=0:
                print(key)
        else:
            break
    # decoder.end_utt()

# Check if the wake word was detected
# if decoder.hyp() is not None:
#     print(f"Wake word detected: {decoder.hyp().hypstr}")
# else:
#     print("Wake word not detected.")


In [2]:
import struct
import wave
def read_file(file_name, sample_rate):
    wav_file = wave.open(file_name, mode="rb")
    channels = wav_file.getnchannels()
    sample_width = wav_file.getsampwidth()
    num_frames = wav_file.getnframes()

    if wav_file.getframerate() != sample_rate:
        raise ValueError("Audio file should have a sample rate of %d. got %d" % (sample_rate, wav_file.getframerate()))
    if sample_width != 2:
        raise ValueError("Audio file should be 16-bit. got %d" % sample_width)
    if channels == 2:
        print("Picovoice processes single-channel audio but stereo file is provided. Processing left channel only.")

    samples = wav_file.readframes(num_frames)
    wav_file.close()

    frames = struct.unpack('h' * num_frames * channels, samples)

    return frames[::channels]

In [7]:
audio_file_path = 'audio/wake_detect_16000_removenoises.wav'
audio = read_file(audio_file_path, 16000)
# keywords = list()
# audio_file_path = 'audio/wake_detect.wav'

# for x in keyword_paths:
#     keyword_phrase_part = os.path.basename(x).replace('.ppn', '').split('_')
#     if len(keyword_phrase_part) > 6:
#         keywords.append(' '.join(keyword_phrase_part[0:-6]))
#     else:
#         keywords.append(keyword_phrase_part[0])

num_frames = len(audio) // porcupine.frame_length
for i in range(num_frames):
    frame = audio[i * porcupine.frame_length:(i + 1) * porcupine.frame_length]
    result = porcupine.process(frame)
    if result >= 0:
        print('Detected', result)

Detected 2


In [1]:
from pydub import AudioSegment

# Load the audio file
audio = AudioSegment.from_wav("audio/wake_detect.wav")

# Set the frame rate to 16000Hz
audio = audio.set_frame_rate(16000)

# Export the audio file with the new frame rate
audio.export("audio/wake_detect_16000.wav", format="wav")

print("Conversion complete.")


Conversion complete.


In [2]:
import noisereduce as nr
import librosa
import soundfile as sf

# Load the audio file
input_file = "audio/prompt.wav"
output_file = "audio/prompt_removenoises.wav"
audio_data, sample_rate = librosa.load(input_file, sr=None)
sample_rate = 44100
# Perform noise reduction
# The noise reduction function takes the audio data and sample rate as input
# Optionally, you can provide a noise clip (a part of the audio containing only noise) for better results
reduced_noise = nr.reduce_noise(y=audio_data, sr=sample_rate)

# Save the cleaned audio file
sf.write(output_file, reduced_noise, sample_rate)

print("Noise reduction complete. Cleaned file saved as", output_file)


Noise reduction complete. Cleaned file saved as audio/prompt_removenoises.wav


In [29]:
from openai import OpenAI
client = OpenAI()
audio_file= open('audio/speak.mp3', "rb")
transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="verbose_json"
)
transcription

Transcription(text='Mình tên là Librarios. Mình là trợ lý ảo của Thư viện HSMU. Cần giúp gì cho bạn?', task='transcribe', language='vietnamese', duration=5.28000020980835, segments=[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 5.0, 'text': ' Mình tên là Librarios. Mình là trợ lý ảo của Thư viện HSMU. Cần giúp gì cho bạn?', 'tokens': [50364, 376, 10116, 256, 5050, 3684, 15834, 5352, 2717, 13, 376, 10116, 3684, 504, 27836, 287, 11822, 220, 40657, 11990, 334, 4228, 1932, 19679, 389, 26693, 52, 13, 383, 18241, 1735, 44492, 22804, 1586, 14647, 30, 50614], 'temperature': 0.0, 'avg_logprob': -0.28208866715431213, 'compression_ratio': 0.9528301954269409, 'no_speech_prob': 0.0314621776342392}])

In [20]:
from openai import OpenAI
client = OpenAI()
audio_file= open(output_file, "rb")
transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="verbose_json"
)
transcription

Transcription(text='you', task='transcribe', language='english', duration=1.5499999523162842, segments=[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 2.059999942779541, 'text': ' you', 'tokens': [50364, 291, 50467], 'temperature': 0.0, 'avg_logprob': -0.9010197520256042, 'compression_ratio': 0.27272728085517883, 'no_speech_prob': 0.950764000415802}])

In [26]:
transcription.segments[0]['no_speech_prob']

0.950764000415802

In [28]:
transcription.language

'english'

In [5]:
# Load the audio file
input_file = "audio/prompt.wav"
output_file = "audio/prompt_removenoises.wav"
audio_data, sample_rate = librosa.load(input_file, sr=None)
sample_rate = 44100
# Extract a noise profile (e.g., first 0.5 seconds of the audio)
noise_profile = audio_data[0:int(0.1 * sample_rate)]

# Perform noise reduction using the noise profile
# y: the audio data to reduce noise from
# sr: sample rate of the audio
# y_noise: a segment of the audio containing only noise for profiling
reduced_noise = nr.reduce_noise(y=audio_data, sr=sample_rate, y_noise=noise_profile)

# Save the cleaned audio file
sf.write(output_file, reduced_noise, sample_rate)

print("Noise reduction complete. Cleaned file saved as", output_file)

Noise reduction complete. Cleaned file saved as audio/prompt_removenoises.wav


In [12]:
import requests

# Assuming you have already set up the API key and endpoint
url = 'https://api.openai.com/v1/audio/transcriptions'

headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
}

data = {
    'audio_url': output_file,  # URL to your audio file
    'model': 'whisper-1',  # or whatever the model identifier is
    # Other parameters as needed
}

response = requests.post(url, headers=headers, json=data)

if response.status_code == 200:
    response_data = response.json()
    transcription = response_data.get('transcription', '')
    # Assuming 'confidence' is a field in the response
    confidence_score = response_data.get('confidence', None)
    
    print(f'Transcription: {transcription}')
    if confidence_score is not None:
        print(f'Confidence Score: {confidence_score}')
else:
    print(f'Error: {response.status_code} - {response.text}')


Error: 400 - {
  "error": {
    "message": "1 validation error for Request\nbody -> file\n  field required (type=value_error.missing)",
    "type": "invalid_request_error",
    "param": null,
    "code": null
  }
}


In [ ]:
import os
from pocketsphinx import Decoder, Config

# Path to the sphinx models directory
model_path = 'sphinx_models'

# Ensure paths are correctly set
hmm_path = os.path.join(model_path, 'cmusphinx-en-us-5.2')
dict_path = os.path.join(model_path, 'cmudict-en-us.dict')

# Configuration for the decoder
config = Config()
config.set_string('-hmm', hmm_path)
config.set_string('-dict', dict_path)
config.set_string('-keyphrase', 'hey assistant')
config.set_float('-kws_threshold', 1e-20)

try:
    # Create a decoder with the specified configuration
    decoder = Decoder(config)
except RuntimeError as e:
    print(f"Failed to initialize PocketSphinx: {e}")
    exit(1)

# Path to your recorded audio file
audio_file_path = 'audio.wav'  # Replace with the path to your recorded audio file

try:
    # Open the audio file and decode it
    with open(audio_file_path, 'rb') as audio_file:
        decoder.start_utt()
        while True:
            buf = audio_file.read(1024)
            if buf:
                decoder.process_raw(buf, False, False)
            else:
                break
        decoder.end_utt()
except Exception as e:
    print(f"Failed to process audio file: {e}")
    exit(1)

# Check if the wake word was detected
if decoder.hyp() is not None:
    print(f"Wake word detected: {decoder.hyp().hypstr}")
else:
    print("Wake word not detected.")

# Print the words recognized during the decoding
for seg in decoder.seg():
    print(f"Segment: {seg.word}")


Failed to initialize PocketSphinx: Failed to initialize PocketSphinx
Wake word not detected.


ERROR: "ps_config.c", line 158: Only one of lm, jsgf, fsg, keyphrase, kws, allphone, lmctl, can be enabled at a time in config


TypeError: 'NoneType' object is not iterable

# YOUTUBE

In [ ]:
from os import system
import speech_recognition as sr
import soundfile as sf
from playsound import playsound
r = sr.Recognizer()
source = sr.Microphone()
import time
import sys
sys.path.append("../")
import API_keys
from openai import OpenAI
client = OpenAI()
i = 0

wake_word = 'apple'
listening_for_wake_word = True

def listen_for_wake_word(audio):
    # print('/////////////////////////', len(audio.frames))
    global listening_for_wake_word, i
    with open('wake_detect.wav', 'wb') as f:
        f.write(audio.get_wav_data())
    f = sf.SoundFile('wake_detect.wav')
    # if f.frames > 10000:
    i += 1
    with open(f'audio/wake_detect_{i}.wav', 'wb') as f:
        f.write(audio.get_wav_data())
    # result = tiny_model.transcribe('wake_detect.wav')
    # text_input = result['text']
    audio_file= open('wake_detect.wav', "rb")
    transcription = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    )
    text_input = transcription.text
    print('/////////////////////////////////', text_input)
    if wake_word in text_input.lower().strip():
        print('Wake word detected.')
        # speak('Listening')
        # listening_for_wake_word = False

def callback(recognizer, audio):
    global listening_for_wake_word
    if listening_for_wake_word:
        listen_for_wake_word(audio)
    else:
        playsound('How can I help you')
        listening_for_wake_word = True
        # return

with source as s:
    r.adjust_for_ambient_noise(s, duration=1)
print('\nSay', wake_word, 'to wake me up. \n')
r.listen_in_background(source, callback)
while True:
    time.sleep(1)


Say apple to wake me up. 

///////////////////////////////// Hẹn đại gia đình video tiếp theo nhé các bạn ạ!
Thank you for waking me up!
///////////////////////////////// 來來來
///////////////////////////////// 見てくれてありがとう!
///////////////////////////////// Ay dun!
///////////////////////////////// Apple.
Wake word detected.
///////////////////////////////// Ơiiiii
///////////////////////////////// Apple.
Wake word detected.
///////////////////////////////// Các bạn hãy đăng kí cho kênh lalaschool Để không bỏ lỡ những video hấp dẫn


KeyboardInterrupt: 

///////////////////////////////// HÃY ĐĂNG KÍ KÊNH NHÉ ĐẠI GIA ĐÌNH ^^
///////////////////////////////// Tạm biệt.
///////////////////////////////// Thank you for watching.


In [ ]:
from playsound import playsound
playsound("wake_detect.wav")

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


In [ ]:
import os

file_name = "wake_detect.wav"
# file_name = "/home/xuanai/LLM/Bot/retrieval-text/audio/wake_detect_147.wav"

file_stats = os.stat(file_name)

print(file_stats)
print(f'File Size in Bytes is {file_stats.st_size}')
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

os.stat_result(st_mode=33204, st_ino=19055436, st_dev=66309, st_nlink=1, st_uid=1000, st_gid=1000, st_size=135212, st_atime=1715784004, st_mtime=1715784004, st_ctime=1715784004)
File Size in Bytes is 135212
File Size in MegaBytes is 0.12894821166992188


In [ ]:
import wave
import contextlib
fname = "audio.wav"
fname = "/home/xuanai/LLM/Bot/retrieval-text/audio/wake_detect_147.wav"
with contextlib.closing(wave.open(fname,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)
    print(duration)

1.509297052154195


In [ ]:
import librosa
librosa.get_duration(filename=fname)

ModuleNotFoundError: No module named 'librosa'

In [ ]:
import soundfile as sf

# fname = "audio.wav"
fname = "prompt.wav"
f = sf.SoundFile(fname)
print('samples = {}'.format(f.frames))
print('sample rate = {}'.format(f.samplerate))
print('seconds = {}'.format(f.frames / f.samplerate))

samples = 0
sample rate = 44100
seconds = 0.0


In [ ]:
import sys
sys.path.append("../")
import API_keys
from Voice_handle import VoiceHandle
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage


llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
function_def = {
    "name": "route",
    "description": "trả lời câu hỏi của người dùng",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "messages": {
                "title": "messages",
                "ouput": 'text',
            }
        },
        "required": ["messages"],
    },
}
system_assistant_prompt = (
    "Bạn là một trợ lí robot thông minh phục vụ trong thư viện HCMUTE, bạn tên là Librarios. Bạn là một robot di động có 2 bánh xe giúp bạn có thể đi lại tự do trong thư viện, "
    "nhờ đó bạn có thể dẫn người dùng đến nơi để những cuốn sách mà sinh viên hay giảng viên mong muốn."
    "Lưu ý: chỉ trả lời những câu hỏi bằng tiếng việt và có nghĩa, nếu không hãy phản hồi là bạn không hiểu câu hỏi."
)

assistant_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_assistant_prompt),
        # MessagesPlaceholder(variable_name="history"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

assistant_chain = (
    assistant_prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)
def getchatresponse(prompt_text:str) -> str:
    global assistant_chain
    response = assistant_chain.invoke([HumanMessage(prompt_text)])['messages'][0]
    return response


In [ ]:
voicehandle = VoiceHandle(wake_words=['porcupine'], get_chat_response_func=getchatresponse)
voicehandle.run()


Say one of  ['porcupine'] to wake me up. 

_______________Call back
Detected word:  -1
_______________Call back
Detected word:  -1


KeyboardInterrupt: 

_______________Call back
Detected word:  -1


In [38]:
class Voice:
    def __init__(self):
        self._listening = False

    @property
    def listening(self):
        return self._listening

    @listening.setter
    def listening(self, value):
        self._listening = False
        self.call_back()
    def dosomething(self, ):
        self.listening = False
    def call_back(self):
        print('The call_back function has been executed')
        # do something

# Example usage
v = Voice()
# v.listening = False  # This will trigger the call_back method

In [42]:
v.listening = True

The call_back function has been executed


In [43]:
v.listening

False

In [10]:
v.dosomething()
print(v.listening)

The call_back function has been executed
False


In [53]:
import sounddevice as sd
import soundfile as sf
def play_wav(file_path):
    data, fs = sf.read(file_path, dtype='float32')
    sd.play(data, fs)
    print(1)

In [54]:
play_wav('/home/xuanai/LLM/Bot/retrieval-text/audio/speak.mp3')
print(1)

1
1


In [9]:
from pydub import AudioSegment

def cut_audio(input_file, output_file, start_time, end_time):
    # Load audio file
    audio = AudioSegment.from_file(input_file)
    
    # Convert start_time and end_time from milliseconds to seconds
    start_time /= 1000
    end_time /= 1000
    
    # Cut audio
    new_audio = audio[start_time * 1000:end_time * 1000]
    
    # Export cut audio to a file
    new_audio.export(output_file, format="wav")

# Thời gian bắt đầu và kết thúc cắt audio (đơn vị: milliseconds)
start_time = 0  # Ví dụ: cắt từ giây thứ 5
end_time = 450   # Ví dụ: cắt đến giây thứ 15

input_file = "/home/xuanai/LLM/Bot/retrieval-text/audio/notifications-sound.mp3"
output_file = "output_audio.wav"

cut_audio(input_file, output_file, start_time, end_time)


In [1]:
import soundfile as sf
import sounddevice as sd
import threading
def callback():
    print('callback')
def play_wav(file_path, callback=None):
    def audio_finished():
        sd.wait()
        if callback:
            callback()

    data, fs = sf.read(file_path, dtype='float32')
    sd.play(data, fs)
    threading.Thread(target=audio_finished).start()

In [2]:
play_wav('/home/xuanai/LLM/Bot/retrieval-text/audio/notifications-sound.mp3', callback)
print(1)

1


callback


In [9]:
import speech_recognition as sr
import soundfile as sf
import sounddevice as sd
import time

def call_back(recognizer, audio):
    print('___Call back: ')
recognizer = sr.Recognizer()
source = sr.Microphone()
with source as s:
    recognizer.adjust_for_ambient_noise(s, duration=1)
stop_listening =  recognizer.listen_in_background(source, call_back)
print('\nSay one of ... to wake me up. \n')
while True:
    time.sleep(1)


Say one of ... to wake me up. 



KeyboardInterrupt: 

In [10]:
stop_listening(wait_for_stop=False)

In [3]:
import urllib
def text_to_speech(url,filename):
    try_times = 3
    while (try_times>0):
        try :
            download_zalo_audio(url,filename)
            return 
        except:
            try_times = try_times -1
            continue
def download_zalo_audio(url,filename):
# Open the URL and read the content
    with urllib.request.urlopen(url) as response:
        audio_data = response.read()
# Write the content to a file
    with open(filename, 'wb') as f:
        f.write(audio_data)

    print("File downloaded successfully as", filename)

In [2]:
import requests
import sys
sys.path.append("../")
import API_keys


# Define the API endpoint
url = "https://api.zalo.ai/v1/tts/synthesize"

# Define the headers
headers = {
    "apikey": API_keys.zalo_api
}

# Define the data payload
data = {
    "input": "Rất nhiều khá ch hàng hỏi chúng tôi vì sao đôla Mỹ lại mất giá.",
    "speaker_id": 3
}

# Make the POST request
response = requests.post(url, headers=headers, data=data)

# Check the response
if response.status_code == 200:
    # Successful request
    print("Response received:")
    print(response.content)
else:
    # Handle errors
    print(f"Request failed with status code {response.status_code}")
    print(response.text)
# text_to_speech(response.content,filename)

Response received:
b'{"error_code":0,"error_message":"Successful.","data":{"url":"https://chunk.lab.zalo.ai/c9289666af0d46531f1c/c9289666af0d46531f1c"}}'


In [8]:
str(response.content["data"])

TypeError: byte indices must be integers or slices, not str